<img align="left" src="https://github.com/Db2-DTE-POC/CPDDVHOL4/blob/main/media/Assets&ArchHeader.jpeg?raw=true">

### Db2 Console Class for Cloud Pak for Data
This Jupyter Notebook contains a reusable Python class library that encapsulates come best practices of how to use the Open APIs that are available in Cloud Pak for Data. Everything in the User Interface is available through an open RESTful Services API. The full set of APIs are documented as part of the Db2 Data Management Console user interface. 

### Where to find this sample online
You can find a copy of this notebook at https://github.com/Db2-DTE-POC/CPDDVHOL4

### First we will import a few helper classes
We need to pull in a few standard Python libraries so that we can work with REST, JSON and a library called Pandas. Pandas lets us work with DataFrames, which are a very powerful way to work with tabular data in Python. 

In [1]:
# Import the class libraries 
import requests
import ssl
import json
from pprint import pprint
from requests import Response
import pandas as pd
import time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from pandas import json_normalize
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

### The Db2 Class
Next we will create a Db2 helper class that will encapsulate the Rest API calls that we can use to directly access the Data Management Console service running on Cloud Pak for Data without having to use the user interface. 

To access the service we need to first authenticate with the service and create a reusable token that we can use for each call to the service. This ensures that we don't have to provide a userID and password each time we run a command. The token makes sure this is secure. 

To initialize the class you require the url address of the Cloud Pak for Data Console as well as the full name of the Data Management Console instance running on Cloud Pak for Data.

Each request is constructed of several parts. First, the URL and the API identify how to connect to the service. Second the REST service request that identifies the request and the options. For example '/metrics/applications/connections/current/list'. And finally some complex requests also include a JSON payload. For example running SQL includes a JSON object that identifies the script, statement delimiters, the maximum number of rows in the results set as well as what do if a statement fails.

In [2]:
# Run the Db2 Class library
# Used to construct and reuse an Autentication Key
# Used to construct RESTAPI URLs and JSON payloads
class Db2Console():
    
    def __init__(self, url, instance, verify = False, proxies=None, ):
        self.DMCInstance = instance
        self.loginurl = url
        self.url = url+'/icp4data-addons/'+instance+'/cpd-instance/dbapi/v4'
        self.proxies = proxies
        self.verify = verify
        self.bearerToken = ''

    def authenticate(self, api, userid, password, profile=""):
        
        credentials = {'username':userid, 'password':password}
        r = requests.post(self.loginurl+api+'/preauth/signin', verify=self.verify, json=credentials, proxies=self.proxies)
        if (r.status_code == 200):
            self.bearerToken = "Bearer " + r.cookies["ibm-private-cloud-session"]
            print('Token Retrieved')
            if profile == "":
                self.headers = {'Content-Type':"application/json", 'Accept':"application/json", 'Authorization': self.bearerToken, 'Cache-Control': "no-cache"}
                return True;
            else: 
                self.headers = {'Content-Type':"application/json", 'Accept':"application/json", 'Authorization': self.bearerToken, 'X-DB-Profile': profile, 'Cache-Control': "no-cache"}
                return True;
        else:
            print ('Unable to authenticate, no bearer token obtained')
        
    def setDatabase(self, profile):
        self.headers = {'Content-Type':"application/json", 'Accept':"application/json", 'Authorization': self.bearerToken, 'X-DB-Profile': profile, 'Cache-Control': "no-cache"}
    
    def getBearerToken(self):
        return self.bearerToken
        
    def printResponse(self, r, code):
        if (r.status_code == code):
            pprint(r.json())
        else:
            print (r.status_code)
            print (r.content)
    
    def getRequest(self, api, json=None):
        return requests.get(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json)

    def postRequest(self, api, json=None):
        return requests.post(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
    
    def deleteRequest(self, api, json=None):
        return requests.delete(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
    
    def putRequest(self, api, json=None):
        return requests.put(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
        
    def getStatusCode(self, response):
        return (response.status_code)

    def getJSON(self, response):
        return (response.json())
    
    def runSQL(self, script, limit=10, separator=';', stopOnError=False):
        sqlJob = {'commands': script, 'limit':limit, 'separator':separator, 'stop_on_error':str(stopOnError)}
        return self.postRequest('/sql_jobs',sqlJob)
        
    def getSQLJobResult(self, jobid):
        return self.getRequest('/sql_jobs/'+jobid)
    
    def getUserPrivilegesAPI(self, profile=''):
        if profile == '' :
            return self.getRequest('/userProfilePrivileges')
        else : 
            return self.getRequest('/userProfilePrivileges/'+profile)
        
    def getUserPrivileges(self):
        r = databaseAPI.getUserPrivilegesAPI()
        if (databaseAPI.getStatusCode(r)==200):
            json = databaseAPI.getJSON(r)
            return pd.DataFrame(json_normalize(json))
        else:
            print(databaseAPI.getStatusCode(r))   
    
    def assignUserPrivileges(self, profile, user):
        json = [{'profileName': profile, 'USER':[user], 'OWNER':[]}]
        return self.postRequest('/userProfilePrivileges?action=assign', json) 
 
    def assignOwnerPrivileges(self, profile, owner):
        json = [{'profileName': profile, 'USER':[], 'OWNER':[owner]}]
        return self.postRequest('/userProfilePrivileges?action=assign', json) 
    
    def addProfilePrivileges(self, profile, name, userType):
        if userType == 'user':
            r = self.assignUserPrivileges(profile, name)
        else:
            r = self.assignOwnerPrivileges(profile, name)
        if (self.getStatusCode(r)==201):
            print(name+' added to: '+profile+" as a new "+userType+".")
        else:
            print(self.getStatusCode(r)) 
        return self.getRequest('/monitor') 
    
    def getConnectionProfilesAPI(self):
        return self.getRequest('/dbprofiles')  
    
    def getConnectionProfiles(self):
        r = self.getConnectionProfilesAPI()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r)
            return pd.DataFrame(json_normalize(json))
        else:
            print(self.getStatusCode(r))  
    
    def getConsoleRepositoryAPI(self):
        return self.getRequest('/repository')    
    
    def getConsoleRepository(self):
        r = self.getConsoleRepositoryAPI()
        if (self.getStatusCode(r)==200):
            json = databaseAPI.getJSON(r) 
            display(pd.DataFrame(json_normalize(json)[['databaseName','status','host','port','collectionCred.user']]).transpose())
        else:
            print(self.getStatusCode(r))     

    def getExportConnectionProfiles(self):
        return self.getRequest('/dbprofiles/transfer/export?exportCred=true')
    
    def postConnectionProfile(self, connectionName, dbName, port, host, userid, password, comment):
        json = {"name":connectionName,"location":"","databaseName":dbName,"dataServerType":"DB2LUW","port":port,"host":host,"URL":"jdbc:db2://"+host+":"+port+"/"+dbName+":retrieveMessagesFromServerOnGetMessage=true;","sslConnection":"false","disableDataCollection":"false","collectionCred":{"securityMechanism":"3","user":userid,"password":password},"operationCred":{"securityMechanism":"3","user":userid,"password":password,"saveOperationCred":"true"},"comment":comment}
        return self.postRequest('/dbprofiles', json)
    
    def putConnectionProfileUpdate(self, connectionName, dbName, port, host, userid, password, comment):
        json = {"name":connectionName,"location":"","databaseName":dbName,"dataServerType":"DB2LUW","port":port,"host":host,"URL":"jdbc:db2://"+host+":"+port+"/"+dbName+":retrieveMessagesFromServerOnGetMessage=true;","sslConnection":"false","disableDataCollection":"false","collectionCred":{"securityMechanism":"3","user":userid,"password":password},"operationCred":{"securityMechanism":"3","user":userid,"password":password,"saveOperationCred":"true"},"comment":comment}
        return self.putRequest('/dbprofiles/'+connectionName, json)
    
    def postTestConnection(self, dbName, port, host, userid, password):
        json = {"name":"","location":"","databaseName":dbName,"dataServerType":"DB2LUW","port":port,"host":host,"URL":"jdbc:db2://"+host+":"+port+"/"+dbName+":retrieveMessagesFromServerOnGetMessage=true;","sslConnection":"false","disableDataCollection":"false","operationCred":{"securityMechanism":"3","user":userid,"password":password}}
        return self.postRequest('/dbprofiles/testConnection', json)
    
    def deleteConnectionProfile(self, connectionName):
        r =  self.deleteRequest('/dbprofiles/'+connectionName)
        if (self.getStatusCode(r)==200):
            print("Deleted connection profile "+connectionName)
        else:
            print(self.getStatusCode(r)) 

    def getMonitoringProfilesAPI(self):
        return self.getRequest('/monitorprofile/front')
    
    def getMonitoringProfiles(self):    
        r = self.getMonitoringProfilesAPI()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r) 
            return pd.DataFrame(json_normalize(json['resources']))
        else:
            print(databaseAPI.getStatusCode(r)) 
    
    def getMonitoringProfile(self, profileID):
        return self.getRequest('/monitorprofile/front/'+profileID)
    
    def displayMonitoringProfile(self, profileID):
        r = self.getMonitoringProfile(profileID)
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r) 
            print(json['base_info'])
            print(json['monitor_config']['common_settings'])
        else:
            print(self.getStatusCode(r))       
     
    def putMonitoringProfile(self, profileID, json):
        return self.putRequest('/monitorprofile/front/'+profileID, json)   
    
    def getProfileIndex(self, profileName):
        r = self.getMonitoringProfiles()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r) 
            profileList = pd.DataFrame(json_normalize(json['resources']))[['name','id']]
            display(profileList)
            profileList.set_index('name',inplace=True)
            try:
                profileIndex = profileList.loc[profileName][0]
            except KeyError:
                profileIndex = 0
                print(profileName + " not found")
            return profileIndex
        else:
            print(self.getStatusCode(r)) 
            
    def runScript(self, profile, user, password, sqlText, limit=10, separator=';', stopOnError=False):
        api = '/v1'
        if self.authenticate(api, user, password, profile):

            runID = self.getJSON(self.runSQL(sqlText, limit, separator, stopOnError))['id'] 

            json = self.getJSON(self.getSQLJobResult(runID))
            while 'results' not in json :
                    json = self.getJSON(self.getSQLJobResult(runID))
            fulljson = json

            while json['results'] != [] or (json['status'] != "completed" and json['status'] != "failed") :
                json = self.getJSON(self.getSQLJobResult(runID))
                while 'results' not in json :
                    json = self.getJSON(self.getSQLJobResult(runID))
                for results in json['results'] :
                    fulljson['results'].append(results)
                time.sleep(1) 
            return fulljson
        else :
            print('Could not authenticate')      
        
    def returnRows(self, json, index):
        for results in json['results']:
            if 'error' in results : 
                print(results['error'])
            elif str(results['index']) == str(index):
                df = pd.DataFrame(results['rows'],columns=results['columns'])
                return df
        
    def displayResults(self, json):
        for results in json['results']:
            print('Statement: '+str(results['index'])+': '+results['command'])
            if 'error' in results : 
                print(results['error'])
            elif 'rows' in results :
                df = pd.DataFrame(results['rows'],columns=results['columns'])
                print(df)
            else :
                print('No errors. Row Affected: '+str(results['rows_affected']))
            print()
            
    # self routine builds up a Data Frame containing the run results as we run workloads across databases
    def appendResults(self, df, profile, json) :

        error = ''
        rows = 0
        if 'error' in json :
            print('SQL Service Failed')
        else :
            for results in json['results']:
                if 'error' in results : 
                    error = results['error']
                if 'rows_affected' in results : 
                    rows = results['rows_affected']
                df = df.append({'profile':profile,'index':results['index'], 'statement':results['command'], 'error':error, 'rows_affected': rows, 'runtime_ms':(results['runtime_seconds']*1000)}, ignore_index=True)
            return df
        
    def runWorkload(self, profileList, scriptList, user, password, profileReps, scriptReps, pause=0) :

        df = pd.DataFrame(columns=['profile', 'index', 'statement', 'error', 'rows_affected', 'runtime_ms'])

        for x in range(0, profileReps):
            print("Running repetition: "+str(x))
            for profile in profileList :
                print("  Running scripts against: "+profile)
                for y in range(0, scriptReps) :
                    print("    Running script repetition: "+str(y))
                    for script in scriptList :
                        json = self.runScript(profile, user, password, script)
                        while 'errors' in json:
                            print('    * Trying again *')
                            json = self.runScript(profile, user, password, script)
                        df = self.appendResults(df, profile, json)
                        time.sleep(pause)

        return df
    
    def displayColumns(self, df):
        print('Available Columns')
        print(', '.join(list(df)))
    
    def getCurrentApplicationsConnectionsAPI(self, includeSystem='true'):
        return self.getRequest('/metrics/applications/connections/current/list?&include_sys='+str(includeSystem))

    def getCurrentApplicationsConnections(self, includeSystem='true'):
        r = self.getCurrentApplicationsConnectionsAPI(includeSystem)
        return self.normalize('resources',r)
    
    def getInflightCountAPI(self, startTime, endTime):
        return self.getRequest('/metrics/statements/inflight_executions/current/list?start='+str(startTime)+'&end='+str(endTime));

    def getInflightCount(self, startTime, endTime):
        r = self.getInflightCountAPI(startTime, endTime)
        return self.normalize('resources', r)
    
    def getInflightCurrentListAPI(self, includeSystem='true'):
        return self.getRequest('/metrics/statements/inflight_executions/current/list?'+'&include_sys='+str(includeSystem));

    def getInflightCurrentList(self, includeSystem='true'):
        r = self.getInflightCurrentListAPI(includeSystem)
        return self.normalize('resources', r)
    
    def getIndividualStatementExecutionAPI(self, startTime, endTime, limit=100, includeSystem='false'):
        return self.getRequest('/metrics/statements/evmon_activity?start='+str(startTime)+'&end='+str(endTime)+'&include_sys='+str(includeSystem)+'&offset=0&limit='+str(limit))

    def getIndividualStatementExecution(self, startTime, endTime, limit=100, includeSystem='false'):
        r = self.getIndividualStatementExecutionAPI(startTime, endTime, limit, includeSystem)
        return self.normalize('resources', r)
    
    def getFilesAPI(self, path):
        return self.getRequest('/home'+path)
    
    def getTablesMetricsAPI(self, startTime, endTime, includeSystem='false'):
        return self.getRequest('/metrics/tables?start='+str(startTime)+'&end='+str(endTime)+'&include_sys='+str(includeSystem));

    def getTablesMetrics(self, startTime, endTime, includeSystem='false'):
        r = self.getTablesMetricsAPI(startTime, endTime, includeSystem)
        return self.normalize('resources', r)
    
    def getAverageResponseTimeAPI(self, startTime, endTime):
        return self.getRequest('/metrics/average_response_time?start='+str(startTime)+'&end='+str(endTime));    
    
    def getAverageResponseTime(self, startTime, endTime):
        r = self.getAverageResponseTimeAPI(startTime, endTime)
        json = databaseAPI.getJSON(r)
        return json['coord_act_lifetime_avg_ms']
    
    def getSchemaSizeAPI(self, startTime, endTime, tabSchema):
        return self.getRequest('/metrics/storage/schemas/'+tabSchema+'/timeseries?start='+str(startTime)+'&end='+str(endTime));
  
    def getSchemaSize(self, startTime, endTime, tabSchema):
        r = self.getSchemaSizeAPI(startTime, endTime, tabSchema)
        return self.normalize('resources', r)
    
    def getSearchViewListAPI(self, searchtext, show_systems="false"):
        return self.getRequest('/admin/schemas/obj_type/view?search_name='+searchtext+'&show_systems='+str(show_systems)+'&rows_return=200');
    
    def getSearchViewList(self, searchtext, show_systems="false"):
        r = self.getSearchViewListAPI(searchtext, show_systems)
        return self.normalize('', r)   
    
    def getSearchTableListAPI(self, searchtext):
        return self.getRequest('/admin/schemas/obj_type/table?search_name='+searchtext+'&show_systems=true&rows_return=100');

    def getSearchTableList(self, searchtext):
        r = self.getSearchTableListAPI(searchtext)
        return self.normalize('', r)   
    
    def getRowsReadAPI(self, startTime, endTime):
        return self.getRequest('/metrics/rows_read?start='+str(startTime)+'&end='+str(endTime));

    def getRowsRead(self, startTime, endTime):
        r = self.getRowsReadAPI(startTime, endTime)
        return self.normalize('timeseries', r)    
    
    def getResponseTimeAPI(self, startTime, endTime):
        return self.getRequest('/metrics/response_time?start='+str(startTime)+'&end='+str(endTime));

    def getResponseTime(self, startTime, endTime):
        r = self.getResponseTimeAPI(startTime, endTime)
        return self.normalize('timeseries', r)   
    
    def getStatementsCountAPI(self, startTime, endTime):
        return self.getRequest('/metrics/statements_count?start='+str(startTime)+'&end='+str(endTime));
    
    def getStatementsCount(self, startTime, endTime):
        r = self.getStatementsCountAPI(startTime, endTime)
        json = databaseAPI.getJSON(r)
        return json['app_act_completed_total']        
    
    def getPackageCacheStatementAPI(self, startTime, endTime, show_systems='true'):
        return self.getRequest('/metrics/statements/package_cache?start='+str(startTime)+'&end='+str(endTime)+'&include_sys='+str(show_systems))
    
    def getPackageCacheStatement(self, startTime, endTime, show_systems='true'):
        r = self.getPackageCacheStatementAPI(startTime, endTime, show_systems='true')
        return self.normalize('resources', r)    
    
    def getCountObjectsAPI(self, obj_type, search_text, rows_return=100, show_systems='false'):     
        return self.getRequest('/admin/schemas/obj_type/'+str(obj_type)+'?search_name='+search_text+'&show_systems='+show_systems+'&rows_return='+str(rows_return));
        
    def countObjects(self, obj_type, search_text, rows_return=100, show_systems='false'):
        r = self.getSearchObjectsAPI(obj_type, search_text, rows_return, show_systems)
        return self.normalize('', r)
    
    def postSearchObjectsAPI(self, obj_type, search_text, rows_return=100, show_systems='false'):     
        json = {"search_name":search_text,"rows_return":rows_return,"show_systems":show_systems,"obj_type":obj_type,"filters_match":"ALL","filters":[]}       
        return self.postRequest('/admin/'+str(obj_type)+'s',json);      
        
    def searchObjects(self, obj_type, search_text, rows_return=100, show_systems='false'):
        r = self.postSearchObjectsAPI(obj_type, search_text, rows_return, show_systems)
        return self.normalize('', r)       
             
    def putFileAPI(self, filename, path):
        with open(filename, 'rb') as f:
            r = requests.post(self.url+'/home_content/path', files={filename: f}, verify = self.verify, headers=self.headers, proxies = self.proxies)
            
    def getTablesInSchemaAPI(self, schema):
        return self.getRequest('/schemas/'+str(schema)+'/tables')
    
    def getTablesInSchema(self, schema):
        r = self.getTablesInSchemaAPI(schema)
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r)
            return pd.DataFrame(json_normalize(json['resources']))
        else:
            print(self.getStatusCode(r))     
    
    def getTableStorageBySchemaAPI(self, include_systems="false"):
        return self.getRequest('/metrics/storage/schemas?end=0&include_sys=''+str(include_systems)'+'&limit=1000&offset=0&start=0')
    
    def getTableStorageBySchema(self, include_systems="false"):
        r = self.getTableStorageBySchemaAPI(include_systems)
        return self.normalize('resources', r)  
    
    def getCurrentPackageCacheListAPI(self, include_systems="false"):
        return self.getRequest('/metrics/statements/package_cache/current/list?include_sys='+str(include_systems))

    def getCurrentPackageCacheList(self, include_systems="false"):
        r = self.getCurrentPackageCacheListAPI(include_systems)
        return self.normalize('resources', r)
    
    def getSchemasAPI(self):
        return self.getRequest('/admin/schemas')    
    
    def getSchemas(self):
        r = self.getSchemasAPI()
        return self.normalize('resources', r)
    
    def normalize(self, keyword, r):
        if (self.getStatusCode(r)==200):
            json = databaseAPI.getJSON(r)
            if keyword == '':
                return pd.DataFrame(json_normalize(json))
            else:
                return pd.DataFrame(json_normalize(json[keyword]))
        else:
            print(self.getStatusCode(r))   
            
    def epochtotimeseries(self, epoch):
        return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch/1000))
    
    def KBtoGB(self, kb):
        return kb/1024/1024

#### Credits: IBM 2019-2021, Peter Kohlmann [kohlmann@ca.ibm.com]